In [32]:
import pandas as pd
import numpy as np

df_ap = pd.read_json('zap_ap.json')


In [33]:
df_ap_clean = df_ap.drop(columns=['descricao', 'suite'])
df_ap_clean[['bairro', 'cidade']] = df_ap_clean['bairro'].str.split(', ', expand=True)
df_ap_clean = df_ap_clean[df_ap_clean['cidade'] == 'Belo Horizonte']
del df_ap_clean['cidade']

In [34]:
def remove_outliers(df, column, z_threshold=10):
    """
    Remove outliers de uma coluna específica de um DataFrame com base em um valor de Z-score.

    Parâmetros:
    df (pd.DataFrame): O DataFrame de entrada.
    column (str): O nome da coluna para detectar e remover outliers.
    z_threshold (float): O limiar do Z-score para definir outliers (padrão é 10).

    Retorna:
    pd.DataFrame: O DataFrame sem os outliers na coluna especificada.
    """
    mean_val = df[column].mean()
    std_dev_val = df[column].std()

    # Calcular o Z-score
    df = df.copy()
    df['z_score'] = (df[column] - mean_val) / std_dev_val

    # Detectar e remover outliers
    df = df[df['z_score'] <= z_threshold]
    del df['z_score']

    return df

In [35]:
# Remover outliers das colunas 'valor', 'condominio' e 'iptu'
df_ap_clean = remove_outliers(df_ap_clean, 'valor', 10)
df_ap_clean = remove_outliers(df_ap_clean, 'condominio', 10)
df_ap_clean = df_ap_clean[ df_ap_clean.index != 1839]     # outlier 

In [36]:
def process(value):
    if isinstance(value, str) and '-' in value:
        # Pegar sempre o primeiro valor antes do hífen
        first_value = value.split('-')[0]
        try:
            return int(first_value)
        except ValueError:
            return np.nan
    else:
        # Caso contrário, converter diretamente para inteiro
        try:
            return int(value)
        except ValueError:
            return np.nan

In [37]:
# Lista de colunas para aplicar a função
columns_to_process = ['area(m2)', 'quartos', 'garagem', 'banheiro']

# Transformar colunas em inteiro 
for column in columns_to_process:
    df_ap_clean[column] = df_ap_clean[column].apply(process)

In [38]:
df_ap_clean

,valor,bairro,endereco,quartos,garagem,banheiro,area(m2),condominio,iptu
0,17000,Santa Lúcia,Rua Zodíaco,1,4,3,230,2800,7977
1,5000,Serra,Rua Itaparica,3,1,3,120,700,1375
2,7500,Savassi,Rua Tomé de Souza,2,3,2,72,1018,475
3,3100,Buritis,Rua Engenheiro Alberto Pontes,3,2,3,90,461,220
4,3300,Buritis,Rua Tereza Mota Valadares,3,2,3,95,700,2852
...,...,...,...,...,...,...,...,...,...
3591,18000,Carmo,Rua Boa Esperança,4,4,4,320,3600,1489
3592,25000,Santo Agostinho,Avenida Barbacena,4,4,5,412,2640,19376
3593,25000,Serra,,4,4,5,530,4000,1800
3595,18000,Santo Agostinho,Rua Ouro Preto,4,4,6,367,4250,2208


In [39]:
df_cobertura = pd.read_json('zap_cob.json')

df_cobertura_clean = df_cobertura.drop(columns=['descricao', 'suite'])
df_cobertura_clean[['bairro', 'cidade']] = df_cobertura_clean['bairro'].str.split(', ', expand=True)
df_cobertura_clean = df_cobertura_clean[df_cobertura_clean['cidade'] == 'Belo Horizonte']
del df_cobertura_clean['cidade']

In [40]:
#Analise de outliers foi feita e não foi encontrado nenhum outlier 

In [41]:
# Lista de colunas para aplicar a função
columns_to_process = ['area(m2)', 'quartos', 'garagem', 'banheiro']

# Transformar colunas em inteiro 
for column in columns_to_process:
    df_cobertura_clean[column] = df_cobertura_clean[column].apply(process)

In [42]:
df_cobertura_clean

,valor,bairro,endereco,quartos,garagem,banheiro,area(m2),condominio,iptu
0,4500,Sion,Avenida dos Bandeirantes,3,3,3,220,1200,423
1,4500,União,Rua Domingos Fernandes,5,2,4,330,600,3300
2,3800,Santo Antônio,,1,2,2,80,300,150
3,3800,Jaraguá,Rua Pedro Dutra,3,2,2,138,385,136
4,2200,Acaiaca,Rua João Lemos,3,1,1,80,135,17
...,...,...,...,...,...,...,...,...,...
518,2500,Manacás,Rua Raul Seixas,2,1,1,100,200,125
519,3500,Santa Cruz,Rua José Cleto,2,3,2,100,400,195
520,7000,Carmo,Rua Boa Esperança,2,2,3,132,1,12
521,2800,Camargos,,3,2,2,80,190,201


In [43]:
df_cobertura_clean['penthouse'] =1
df_ap_clean['penthouse'] = 0

In [44]:
df = pd.concat([df_ap_clean, df_cobertura_clean], ignore_index=True)
df

,valor,bairro,endereco,quartos,garagem,banheiro,area(m2),condominio,iptu,penthouse
0,17000,Santa Lúcia,Rua Zodíaco,1,4,3,230,2800,7977,0
1,5000,Serra,Rua Itaparica,3,1,3,120,700,1375,0
2,7500,Savassi,Rua Tomé de Souza,2,3,2,72,1018,475,0
3,3100,Buritis,Rua Engenheiro Alberto Pontes,3,2,3,90,461,220,0
4,3300,Buritis,Rua Tereza Mota Valadares,3,2,3,95,700,2852,0
...,...,...,...,...,...,...,...,...,...,...
4107,2500,Manacás,Rua Raul Seixas,2,1,1,100,200,125,1
4108,3500,Santa Cruz,Rua José Cleto,2,3,2,100,400,195,1
4109,7000,Carmo,Rua Boa Esperança,2,2,3,132,1,12,1
4110,2800,Camargos,,3,2,2,80,190,201,1


In [45]:
def remove_duplicates_with_priority(df):
    """
    Remove duplicatas de um DataFrame com base em colunas específicas,
    mantendo as linhas onde a coluna 'penthouse' é 1.
    
    """
    # Ordenar para que as linhas com 'penthouse' == 1 venham primeiro
    df = df.sort_values(by='penthouse', ascending=False)

    # Remover duplicatas mantendo a primeira ocorrência (onde 'penthouse' == 1)
    df = df.drop_duplicates(subset=['valor', 'bairro', 'endereco','quartos','garagem','banheiro','area(m2)','condominio','iptu'])

    return df

In [46]:
#Removendo os valores que tem TODOS os dados iguais 
df = remove_duplicates_with_priority(df)


In [47]:
#Conferindo se tem algum dado repetido 
df.duplicated(subset=['valor', 'bairro', 'endereco','quartos','garagem','banheiro','area(m2)','condominio','iptu'], keep=False).value_counts()

False    3278
Name: count, dtype: int64

In [48]:
# Renomeando colunas 
df = df.rename(columns={
    'valor': 'price',
    'bairro': 'regionName',
    'endereco': 'address',
    'quartos': 'rooms',
    'garagem': 'parking',
    'banheiro': 'toilets',
    'area(m2)': 'size',
    'condominio': 'condo'
})


In [49]:
df

,price,regionName,address,rooms,parking,toilets,size,condo,iptu,penthouse
4111,3500,Santa Helena (Barreiro),Rua Maria Amélia Mello,3,2,3,160,850,0,1
3597,1600,Pousada Santo Antônio,Rua Ernesto Tognolo,2,1,1,87,450,118,1
3589,4500,Sion,Avenida dos Bandeirantes,3,3,3,220,1200,423,1
3590,4500,União,Rua Domingos Fernandes,5,2,4,330,600,3300,1
3591,3800,Santo Antônio,,1,2,2,80,300,150,1
...,...,...,...,...,...,...,...,...,...,...
519,2500,Coração de Jesus,Rua Perdigão Malheiros,3,1,3,121,410,218,0
520,1350,Jardim Leblon,Rua Coronel Antônio Lopes Coelho,2,1,1,50,280,49,0
521,2650,Barro Preto,,3,2,2,80,620,255,0
522,3000,Buritis,Rua Eli Seabra Filho,2,1,2,67,420,150,0


In [50]:
#Ajustando o IPTU para valor mensal 
df['iptu'] = (df['iptu']/12).astype(int)

In [51]:
df

,price,regionName,address,rooms,parking,toilets,size,condo,iptu,penthouse
4111,3500,Santa Helena (Barreiro),Rua Maria Amélia Mello,3,2,3,160,850,0,1
3597,1600,Pousada Santo Antônio,Rua Ernesto Tognolo,2,1,1,87,450,9,1
3589,4500,Sion,Avenida dos Bandeirantes,3,3,3,220,1200,35,1
3590,4500,União,Rua Domingos Fernandes,5,2,4,330,600,275,1
3591,3800,Santo Antônio,,1,2,2,80,300,12,1
...,...,...,...,...,...,...,...,...,...,...
519,2500,Coração de Jesus,Rua Perdigão Malheiros,3,1,3,121,410,18,0
520,1350,Jardim Leblon,Rua Coronel Antônio Lopes Coelho,2,1,1,50,280,4,0
521,2650,Barro Preto,,3,2,2,80,620,21,0
522,3000,Buritis,Rua Eli Seabra Filho,2,1,2,67,420,12,0


In [52]:
df['iptuPlusCondominium'] = df['condo'] + df['iptu']

In [53]:
df_final = df.copy()

In [54]:
df_final

,price,regionName,address,rooms,parking,toilets,size,condo,iptu,penthouse,iptuPlusCondominium
4111,3500,Santa Helena (Barreiro),Rua Maria Amélia Mello,3,2,3,160,850,0,1,850
3597,1600,Pousada Santo Antônio,Rua Ernesto Tognolo,2,1,1,87,450,9,1,459
3589,4500,Sion,Avenida dos Bandeirantes,3,3,3,220,1200,35,1,1235
3590,4500,União,Rua Domingos Fernandes,5,2,4,330,600,275,1,875
3591,3800,Santo Antônio,,1,2,2,80,300,12,1,312
...,...,...,...,...,...,...,...,...,...,...,...
519,2500,Coração de Jesus,Rua Perdigão Malheiros,3,1,3,121,410,18,0,428
520,1350,Jardim Leblon,Rua Coronel Antônio Lopes Coelho,2,1,1,50,280,4,0,284
521,2650,Barro Preto,,3,2,2,80,620,21,0,641
522,3000,Buritis,Rua Eli Seabra Filho,2,1,2,67,420,12,0,432


In [55]:
df_final.drop(columns=['condo', 'iptu'], inplace=True)

In [56]:
df_final.to_json('dados/zap.json', orient='records', force_ascii=False, indent=4)